In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
market_df =pd.DataFrame({
    'MarketID' : [1, 2, 3],
    'Market Name' : ['Market A', 'Market B', 'Market C'],
    'Market Region' : [ 'Region X', 'Region Y', 'Region Z'],
    'Market Country' : ['Country 1', 'Country 2', 'Country 3'],
})

In [3]:
product_df = pd.DataFrame({
    'Product ID' : [1, 2, 3],
    'Product Name' : ['product A', 'product B', 'product C'],
    'Category': ['category X', 'category Y', 'category Z'],
    'Brand' : ['brand 1', 'brand 2','brand 3'],
})

In [4]:
customer_df = pd.DataFrame({
    'Customer ID' : [1, 2, 3],
    'Customer Name' : ['customer 1', 'customer 2', 'customer 3'],
    'Customer Address' : ['Address 1', 'Address 2', 'Address 3'],
    'Customer Segment' : ['Segment 1', 'Segment 2', 'Segment 3'],
})
   

In [5]:
time_df = pd.DataFrame({
      'Date ID' : [1,2,3],
    'Date' : ['01-01-2023', '02-01-2023', '03-01-2023'],
    'Month' : ['January', 'January', 'January'],
    'Quarter' : ['Q1', 'Q1', 'Q1'],
    'Year' : [2023,2023,2023],
})


In [6]:
pricing_df = pd.DataFrame({
    'Pricing ID' : [1, 2, 3],
    'Product ID' : [1, 2, 3],
    'Market ID' : [1, 2, 3],
    'price' : [10.99, 19.99, 15.99]
})
# I have taken a random set of prices just for example

In [7]:
inventory_df = pd.DataFrame({
    'Inventory ID' : [1, 2, 3],
    'Product ID' : [1, 2, 3],
    'Market ID' : [1, 2, 3],
    'Quantity' : [100, 200, 150]
})
# I have taken a random set of numbers for quantity 

In [8]:
sales_df = pd.DataFrame({
    'Sales ID' : [1, 2, 3],
    'Product ID' : [1, 2, 3],
    'Customer ID' : [1, 2, 3],
    'Market ID' : [1, 2, 3],
    'Date ID' : [1, 2, 3],
    'Quantity ID' : [5, 10, 7],
    'Revenue' : [54.95,199.99,111.93]
})

In [ ]:
engine = create_engine('sqlite:///data_warehouses.db')

market_df.to_sql('Market', engine, index=False, if_exists='replace')
product_df.to_sql('Product', engine, index=False, if_exists='replace')
customer_df.to_sql('Customer', engine, index=False, if_exists='replace')
time_df.to_sql('Time', engine, index=False, if_exists='replace')
pricing_df.to_sql('Price', engine, index=False, if_exists='replace')
inventory_df.to_sql('Inventory', engine, index=False, if_exists='replace')

# Got an error ('Database disk image malformed')
#This could be because of the version incompatibility or data corruption
#In these type of cases, we have to 
#recover the database ('sqlite3 data_warehouse.db ".recover" | sqlite3 recovered.db')
#extract data from it('sqlite3 recovered.db .dump > data.sql')


In [ ]:
#To determine if higher-priced items were more prevalent in some markets,
#we can calculate the average price for each market and compare it across different markets. 

query = '''
    SELECT Market."Market Name", AVG(Price.price) AS AvgPrice
    FROM Market
    INNER JOIN Price ON Market.MarketID = Price.MarketID
    GROUP BY Market."Market Name"
    ORDER BY AvgPrice DESC
'''

result = pd.read_sql_query(query, engine)
print("Markets sorted by average price:")
print(result)


In [ ]:
#To determine whether inventory should be reallocated or prices adjusted based on location,
#we can analyze the relationship between inventory levels and prices in different markets. 
query = '''
    SELECT Inventory."Market ID",
           AVG(Inventory.Quantity) AS AvgQuantity,
           AVG(Price.price) AS AvgPrice
    FROM Inventory
    INNER JOIN Price ON Inventory."Market ID" = Price."Market ID"
    GROUP BY Inventory."Market ID"
'''

result = pd.read_sql_query(query, engine)


for _, row in result.iterrows():
    market_id = row['Market ID']
    avg_quantity = row['AvgQuantity']
    avg_price = row['AvgPrice']
    
    if avg_quantity < 100 and avg_price > 15.00:
        print(f"Inventory reallocation needed for Market {market_id}")
    elif avg_quantity > 200 and avg_price < 10.00:
        print(f"Price adjustment needed for Market {market_id}")
    else:
        print(f"No action needed for Market {market_id}")
